# Assignment 1: Transformations


TEAM-ID: 16 <br>
TEAM-NAME: spacex  <br>
YOUR-ID:   <br>
YOUR-NAME:   <br> 

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################
# Replace "pass" statement with your code
color_raw = o3d.io.read_image("color.jpg")
depth_raw = o3d.io.read_image("depth.png")
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(color_raw, depth_raw)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
o3d.io.write_point_cloud("scene.pcd", pcd)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

True

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [3]:
#uncomment the following and add input parameters if any
def one_one(file):
    pcd = o3d.io.read_point_cloud(file)
    o3d.visualization.draw_geometries([pcd])
one_one("scene.pcd")

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [4]:
def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) Afther picking points, press q for close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

In [5]:
# pick points not working for me rn
pcd = o3d.io.read_point_cloud("scene.pcd")
picked = pick_points(pcd)


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) Afther picking points, press q for close the window
[Open3D INFO] Picked point #199260 (-0.15, -0.95, -3) to add in queue.



In [6]:
print(picked)
picked_coord = pcd.points[picked[0]]

[199260]


In [7]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code


mesh_box = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
mesh_box.compute_vertex_normals()
mesh_box.paint_uniform_color([0.9, 0.1, 0.1])
box = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh_box, number_of_points=100500, use_triangle_normal=False, seed=- 1)
center = o3d.geometry.PointCloud.get_center(box)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame( size=1.2, origin=[-0.5, -0.5, -0.5])
frame = o3d.geometry.TriangleMesh.sample_points_uniformly(mesh_frame, number_of_points=15000, use_triangle_normal=False, seed=- 1)
pcd = frame+ box
o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("cube.pcd", pcd)


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


True

### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [8]:
# ground coordinates = picked_coord - origin_coord
gx = picked_coord[0] - 0.5
gy = picked_coord[1] - 0.5
gz = picked_coord[2] - 0.5

In [9]:
cube = o3d.io.read_point_cloud("cube.pcd")
scene = o3d.io.read_point_cloud("scene.pcd")
o3d.visualization.draw_geometries([cube,scene])

In [10]:
# def rotate(cube,scene):
#     R = cube.get_rotation_matrix_from_xyz((np.pi/6,np.pi/2,np.pi/4))
#     cube.rotate(R, center=(0,0,0))
#     o3d.visualization.draw_geometries([cube,scene])

In [11]:
# working cube comes in rotated geometry, notice diff with default
# rotate(cube,scene)

In [12]:
# working, make it like animation or can keep like that
def custom_draw_geometry_with_rotation(cube,scene):

    def rotate_view(vis):
        R = cube.get_rotation_matrix_from_xyz((np.pi/4,np.pi/2,np.pi/6))
        cube.rotate(R, center=(0,0,0))
#         ctr = vis.get_view_control()
#         ctr.rotate(10.0, 0.0)
        return True

    o3d.visualization.draw_geometries_with_animation_callback([cube,scene],
                                                              rotate_view)

In [13]:
custom_draw_geometry_with_rotation(cube,scene)

In [15]:
#uncomment the following and add input parameters if any
# two_one():
    

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [16]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [17]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code
from scipy.spatial.transform import Rotation as R
def getEuler(M_given):
    beta = np.arctan2(np.sqrt(pow(M_given[2][0],2)+  pow(M_given[2][1],2)), M_given[2][2])
    alpha = np.arctan2(M_given[1][2]/np.sin(beta) , M_given[0][2]/np.sin(beta))
    gamma = np.arctan2(M_given[2][1]/np.sin(beta) , -1*M_given[2][0]/np.sin(beta))
    return alpha,beta,gamma

def reconstructMat(alpha,beta,gamma):
    mat = np.ndarray([3,3])
    mat[0][0] = np.cos(alpha)*np.cos(beta)*np.cos(gamma) - np.sin(alpha)*np.sin(gamma)
    mat[0][1] = -1*np.cos(alpha)*np.cos(beta)*np.sin(gamma) - np.sin(alpha)*np.cos(gamma)
    mat[0][2] = np.cos(alpha)*np.sin(beta)
    mat[1][0] =  np.sin(alpha)*np.cos(beta)*np.cos(gamma) + np.cos(alpha)*np.sin(gamma)
    mat[1][1] = -1*np.sin(alpha)*np.cos(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma)
    mat[1][2] = np.sin(alpha)*np.sin(beta)
    mat[2][0] = -1*np.sin(beta)*np.cos(gamma)
    mat[2][1] = np.sin(beta)*np.sin(gamma)
    mat[2][2] = np.cos(beta)
    return mat

def check(M_given):
    alpha,beta,gamma = getEuler(M_given)
    mat = reconstructMat(alpha,beta,gamma)
    print("from Scrath")
    print("\n Reconstructed angles", alpha," ",beta," ",gamma)
    print("\n Reconstructed Matrix")
    print(mat)
    # checking
    r = R.from_euler('zyx', [[alpha,beta,gamma]], degrees = False)
    print("\n Checking from scipy: Reconstructed matrix")
    print(r.as_matrix())
    return [alpha,beta,gamma],mat

print("* M_given *")
print(M_given)
angles1,M_recovered = check(M_given)
print("\n * N_given *")
print(N_given)
angles2,N_recovered = check(N_given)


##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

* M_given *
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542533  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]
from Scrath

 Reconstructed angles 0.14718877706846195   1.2700866309259595   0.18001508686283857

 Reconstructed Matrix
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

 Checking from scipy: Reconstructed matrix
[[[ 0.29299542 -0.04343979  0.95512652]
  [ 0.31344903  0.94812301 -0.05303262]
  [-0.90327371  0.3149218   0.29141186]]]

 * N_given *
[[ 0.         -0.17364818  0.98480775]
 [ 0.          0.98480775  0.17364818]
 [-1.          0.          0.        ]]
from Scrath

 Reconstructed angles 0.17453292552956243   1.5707963267948966   0.0

 Reconstructed Matrix
[[ 6.03020831e-17 -1.73648178e-01  9.84807753e-01]
 [ 1.06328843e-17  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00  0.00000000e+00  6.12323400e-17]]

 Checking from scipy: Reconstructed matrix
[[[ 1.66533454e-16 -2.77555756

Here, we see that the reconstruction from scratch matches the M_given and N_given more precisely that that from scipy

## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: *We used np.arctan2 fucntion since it returns array of angles in radians, in the range [-pi, pi] and we required the range to be -pi to pi in euler angles. On the other hand np.arctan would return in range [-pi/2, pi/2] which was not what we required.

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [18]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(angles1))

My Euler angles for case 1 are[0.14718877706846195, 1.2700866309259595, 0.18001508686283857]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [19]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:    *No problem, could recover accurately by coding from scrath but didnt recover by using scipy function*

### Repeat the above for Case 2.

In [20]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(angles2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[0.17453292552956243, 1.5707963267948966, 0.0]
For case 2, it is True I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: *I dont see any problem. Was able to recover by coding from scrath.*
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: *your answer here.*

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: *your answer here.*
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: *your answer here.*
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: *your answer here.*


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{Ans: Your set of equations here} $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = your answer here.* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = your answer here.*

In [21]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [22]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code
import numpy as np
from scipy.optimize import fsolve
def solve(arr):
    x_1 = np.array([0,np.sqrt(3),0])
    x_2 = np.array([1,1,1])
    alpha,beta,gamma = arr[0],arr[1],arr[2]
    R = np.array([[np.cos(alpha)*np.cos(beta),np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma),np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)],
              [np.sin(alpha)*np.cos(beta),np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma),np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
              [-1*np.sin(beta),np.cos(beta)*np.sin(gamma),np.cos(beta)*np.cos(gamma)]])
    
    x_1 =  x_1.T.reshape(3,1)
    x_2 = x_2.T.reshape(3,1)
    
    return (x_2 - np.dot(R,x_1)).T.reshape(3)
sol = fsolve(solve,[0,0,0])
a1,b1,c1 = sol[0],sol[1],sol[2]
print(a1,b1,c1)

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

-0.7166830461925598 100.6277628440765 0.6188093556149915


In [23]:

x_2_obtained_case1 = reconstructMat(a1,b1,c1) @ x_1.T #TODO: replace r_mat
print(x_2_obtained_case1)

[0.1727979  1.72067202 0.09710139]


^ getting x1 :( 

### Visualising 
Analytical solution <br>
Our initial vector is [0,root(3),0] which will lie on y axis on root(3) <br>
Now we rotate it by yamma, beta, alpha to get vector [1,1,1] <br>
We know the components of vector x_2 on: <br> 
y_axis > 1-0 = cos(yamma) <br>
=> cos(yamma) = 1 <br>
=> yamma = 0 rad <br>
on y_axis we have, <br>
cos(beta) = 1- root(3) <br>
=> beta = cos-1(1-root(3))  <br>
=> beta = np.arccos(1-np.sqrt(3))*180/np.pi = beta = 137.0585971351201  rad <br>
Now similarly,  alpha = 0 rad <br>
(since cos(alpha) = 1) <br>

 





In [24]:
b2 = np.arccos(1-np.sqrt(3))*180/np.pi
a2 = 0
c2 = 0

In [25]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = reconstructMat(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = reconstructMat(a2,b2,c2) @ x_2.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [26]:
print(x_2)
print(x_2_obtained_case1)
print(x_2_obtained_case2)

[1 1 1]
[0.1727979  1.72067202 0.09710139]
[-0.53253797  1.          1.31011576]


In [27]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is False that I could end up at (1,1,1) after rotation.
For case 2, it is False that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
pass
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: *your answer here.*
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: *your answer here.*
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [ ]:
#uncomment the following and add input parameters if any
#two_four()